In [1]:
# import tensorflow library
import tensorflow as tf
import time
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
# one_hot key implies lables in onehot encoding
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

In [3]:
# Network Parameters
n_input = 784 # 28*28
n_classes = 10 # (0-9 digits)
dropout = 0.75 # What use?

In [4]:
# define placeholders
x = tf.placeholder(tf.float32, shape=(None, n_input))
y = tf.placeholder(tf.float32, shape=(None, n_classes))

In [5]:
# miscill... functions
def define_variable(shape, name): 
    return tf.Variable(tf.truncated_normal(shape, name = name))

def conv2d(x, W, b):
    x = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [6]:
weights = {
    'wc1': define_variable([5, 5, 1, 32], 'wc1'), # 5x5 conv, 1 input, 32 outputs
    'wc2': define_variable([5, 5, 32, 64], 'wc2'), # 5x5 conv, 32 inputs, 64 outputs
    'wfc1': define_variable([7*7*64, 2048], 'wfc1'), # fully connected, 7*7*64 inputs, 2048 outputs
    'wfc2': define_variable([2048, 1024], 'wfc2'), # fully connected, 2048 inputs, 1024 outputs
    'out': define_variable([1024, n_classes], 'out') # 1024 inputs, 10 outputs (class prediction)
}

In [7]:
biases = {
    'bc1': define_variable([32], 'bc1'),
    'bc2': define_variable([64], 'bc2'),
    'bfc1': define_variable([2048], 'bfc1'),
    'bfc2': define_variable([1024], 'bfc2'),
    'out': define_variable([n_classes], 'out')
}

In [8]:
def main_net(x, weights, biases, dropout = dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1)

    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2)
    
    print conv2
    
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wfc1'].get_shape().as_list()[0]])
    
    # fully connected layer 1
    fc1 = tf.add(tf.matmul(fc1, weights['wfc1']), biases['bfc1'])
    fc1 = tf.nn.relu(fc1)
    
    # fully connected layer 2
    fc2 = tf.add(tf.matmul(fc1, weights['wfc2']), biases['bfc2'])
    fc2 = tf.nn.relu(fc2)

    # Output, class prediction
    out = tf.add(tf.matmul(fc2, weights['out']), biases['out'])
    return out

In [9]:
# Construct model
pred = main_net(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)


In [10]:
# Launch the graph
with tf.Session() as sess:
    t = time.time()
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x, y: batch_y})
            print("loss= {:.6f}".format(loss) + ", Accuracy= {:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")
    print t
    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256], y: mnist.test.labels[:256]}))

loss= 238107.031250, Accuracy= 0.46875
loss= 148056.953125, Accuracy= 0.55469
loss= 80411.046875, Accuracy= 0.71094
loss= 37589.507812, Accuracy= 0.87500
loss= 28106.335938, Accuracy= 0.85156
loss= 69239.265625, Accuracy= 0.78125
loss= 15126.808594, Accuracy= 0.90625
loss= 17448.589844, Accuracy= 0.86719
loss= 12537.723633, Accuracy= 0.92969
loss= 24409.667969, Accuracy= 0.82812
loss= 10591.248047, Accuracy= 0.92188
loss= 8869.946289, Accuracy= 0.94531
loss= 30581.945312, Accuracy= 0.82812
loss= 18343.697266, Accuracy= 0.89844
loss= 15837.876953, Accuracy= 0.89844
loss= 8268.785156, Accuracy= 0.96875
loss= 39679.828125, Accuracy= 0.83594
loss= 5669.328125, Accuracy= 0.94531
loss= 11714.302734, Accuracy= 0.91406
loss= 12194.192383, Accuracy= 0.92969
loss= 13985.338867, Accuracy= 0.93750
loss= 9119.533203, Accuracy= 0.92969
loss= 18172.429688, Accuracy= 0.92188
loss= 12664.756836, Accuracy= 0.91406
loss= 9971.997070, Accuracy= 0.91406
loss= 16776.730469, Accuracy= 0.89844
loss= 2138.8510